In [1]:
import numpy as np
import time
import copy

class Tree:
    # classe Tree cria um nó quando é chamada
    def __init__(self, data):
        self.children = []
        self.parents = []
        self.data = data

    def __str__(self):
    # quando print é chamada para um objeto do tipo Tree, ele exibe o dado do nó 
    # atual e dos filhos apenas desse nó
        children_data = []
        for child in self.children:
            child_data = str(child.data)
            children_data.append(child_data)
        a = "Data: "+str(self.data)+"\n"+"Children: "+str(children_data)
        return a

def verifica_validade(x,y):
    #verifica se estao na mesma linha
    if x//8==y//8:
        return False
    #verifica se estao na mesma coluna
    elif x%8==y%8:
        return False
    #verifica se estao na mesma diagonal principal (\)
    elif abs(x-y)%9==0 and max(x,y)%8>min(x,y)%8:
        return False
    #verifica se estao na mesma diagonal secundaria (/)
    elif abs(x-y)%7==0 and max(x,y)%8<min(x,y)%8:
        return False
    #se todas as verificacoes passaram a posicao e valida
    else:
        return True

def cria_arvore(node_pai, indice_pai, nivel):
    #a arvore é criada no nivel 1 e deve ter no máximo 8 níves. logo, se nivel==9, a criação da árvore deve cessar
    if nivel==9:
        return None
    #verifica-se se o pai pode ter filhos. caso a posição de um nó seja inválida, tal nó é armazenado com um *, o que torna seu tipo de dado
    #uma string.
    if type(node_pai.data) != str:
        #cria-se filhos para o nó pai atual com índices maiores que o do nó pai. Logo, a árvore é criada apenas com numeração ascendente
        #Isto é: 0->10->25->...
        #isso elimina a verificação repetida em combinações já testadas, como 0->25->10->...
        for indice_atual in range(indice_pai+1,64):
            #cria-se um nó vazio
            node_atual = Tree(None)
            #copia-se os nós do pai para o nó recém-criado
            node_atual.parents=copy.deepcopy(node_pai.parents)
            #adiciona-se o pai na lista de pais do nó atual
            node_atual.parents.append(indice_pai)
            valido=True
            #verifica-se se a posição do nó atual é válida em relação a cada um dos nós pai que o precederam na árvore
            for parent in node_atual.parents:
                if not verifica_validade(parent,indice_atual):
                    valido=False
                    break
            #caso o nó seja válido, adiciona-se o índice atual a ele
            if valido:
                node_atual.data = indice_atual
            else:
                node_atual.data=str(indice_atual)+"*"
            #após a criação do nó, ele é adicionado a lista de filhos de seu pai
            node_pai.children.append(node_atual)

            #incrementa-se o contador de nível da árvore
            nivel += 1
            #chama-se a função de forma recursiva para criar o próximo nível da árvore a partir do nó atual
            cria_arvore(node_atual, indice_atual, nivel)

def inicia_arvore():
    #visto que o primeiro nível da árvore não segue o mesmo padrão que os demais, ele é criado fora da função recursiva
    raiz = Tree(None)
    for i in range(64):
        node = Tree(i)

        raiz.children.append(node)

        cria_arvore(node, i, 1)
    
    return raiz

In [25]:
def busca_em_profundidade_preliminar(no_inicial):
    pilha = []
    busca_em_profundidade_recursivo(no_inicial, pilha)

def busca_em_profundidade_recursivo(no_referencia, pilha):
    pilha.append(no_referencia)
    for no_filho in no_referencia.children:
        busca_em_profundidade_recursivo(no_filho,pilha)
    if (len(pilha)==9 and pilha[-1]==63): #or (len(pilha)>9)
        print(f"Tamanho da pilha: {len(pilha)}")
        pilha_para_printar = []
        for nozinho in pilha[1:]:
            pilha_para_printar.append(nozinho.data)
        print(pilha_para_printar)
        print("-"*10)
    pilha.pop(pilha.index(no_referencia))

In [7]:
raiz = inicia_arvore()

In [26]:
busca_em_profundidade_preliminar(raiz)

In [24]:
lista = [1,2,4,5,6]
print(lista[-1])
print(lista)

6
[1, 2, 4, 5, 6]


In [1]:
import numpy as np 
import time

In [6]:
lista = []
for i in range(10):
    lista.append([i,i*2])
print(lista)

[[0, 0], [1, 2], [2, 4], [3, 6], [4, 8], [5, 10], [6, 12], [7, 14], [8, 16], [9, 18]]


In [3]:
def verifica_validade(x,y):
    #verifica se estao na mesma linha
    if x//8==y//8:
        return False
    #verifica se estao na mesma coluna
    elif x%8==y%8:
        return False
    #verifica se estao na mesma diagonal principal (\)
    elif abs(x-y)%9==0 and max(x,y)%8>min(x,y)%8:
        return False
    #verifica se estao na mesma diagonal secundaria (/)
    elif abs(x-y)%7==0 and max(x,y)%8<min(x,y)%8:
        return False
    #se todas as verificacoes passaram a posicao e valida
    else:
        return True



In [4]:
lista=[[0, 6], [0, 32], [0, 36], [27, 20], [13, 4], [29, 50], [57, 30], [23, 41], [41, 55], [14, 16], [2, 59], [0, 61]]
for listinha in lista:
    print(verifica_validade(listinha[0], listinha[1]))

False
False
False
False
False
False
True
True
True
True
True
True


In [9]:
s="pikachu"
if type(s)==str:
    print("aspas funcionou")

aspas funcionou


In [8]:
print(type(s))

<class 'str'>


In [10]:
for element in lista:
    print(element[1])

0
2
4
6
8
10
12
14
16
18


In [5]:
for child in lista:
    print(child)

1
3
4
5
6
7
8
9
10


In [2]:
print("helloword")

helloword


In [11]:
x = [1, 3, 4, 5, 6, 7, 8, 9, 10]
x.append(11)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [22]:
print(f"Oi Leo bjs{x[0]} {x[1]} {x[2]}")

Oi Leo bjs1 3 4


In [31]:
s=input()

In [35]:
print(s)

12 13 14 15


In [37]:
v = s.split(" ")
print(v)
print(type(v))

['12', '13', '14', '15']
<class 'list'>


In [41]:
u=[]
print("Dentro do for")
for element in v:
    print(element)
    u.append(int(element))
print("-"*10)
print(element)
print(u)
print(type(u))
print(u[0])
print(type(u[0]))

Dentro do for
12
13
14
15
----------
15
[12, 13, 14, 15]
<class 'list'>
12
<class 'int'>


In [30]:
s_int = float(s)
print(type(s_int))
print(s_int)

<class 'float'>
12.0


In [45]:
print(range(50,100))
print(type(range(50,100)))

range(50, 100)
<class 'range'>


In [48]:
for i in range(10):
    if i%2==0:
        print(i)
    elif i%3==0:
        print(i*10)
    else:
        print(i*100)

0
100
2
30
4
500
6
700
8
90


In [59]:
def soma(a,b):
    return a+b

In [60]:
def mul(a,b):
    return a*b

In [61]:
def div(a,b):
    return a/b

In [68]:
dict = {
    "A": mul,
    "B": soma,
    "C": div
    }

In [71]:
print(dict["C"](2,2))

1.0


In [72]:
P=print

In [73]:
P("Hello World!")

Hello World!


In [78]:
x,y,z=0,1,2

In [79]:
x,y=y,x

In [80]:
print(x)
print(y)
print(z)

1
0
2


In [82]:
x=0
while(x<10):
    print(x)
    x+=1

0
1
2
3
4
5
6
7
8
9


In [94]:
a = True
b = False
c = 124134314123
d = 0

In [98]:
if (not c or a) and (not d):
    print("A")

A


In [90]:
print(type(c))

<class 'int'>


In [99]:
np.log2(4)

2.0

In [133]:
A = np.array([[0,1,2,3],
              [3,4,5,6],
              [8,9,0,1]])

In [135]:
print(A)

[[0 1 2 3]
 [3 4 5 6]
 [8 9 0 1]]


In [136]:
print(np.shape(A))

(3, 4)


In [132]:
np.ones((2,3))

array([[1., 1., 1.],
       [1., 1., 1.]])

In [130]:
print(A)

[[0 1 2 3]]


In [109]:
x = "abcdefghi"

In [126]:
a = x.index("d")
x = x[:a]+"BLA"+x[a:]

In [127]:
print(x)

jklabcBLAdBLAefghijkljkl
